# Advanced Query

The following example shows how to write an advanced query using magic command that allows you to run queries with minimal code and visualize the results.

## TCGA Query 
First query against a 3rd party dataset, this being the Somatic_Mutation table provided by ISB-CGC.


In [ ]:
%%bigquery tcga
WITH genes AS (
  SELECT * FROM UNNEST([
      'ABL1',   'AKT1',    'ALK',    'APC',
       'BRAF',   'CDH1',    'CDKN2A', 'CSF1R',
       'CTNNB1', 'EGFR',    'ERBB2',  'ERBB3',
       'ESR1',   'FBXW7',   'FGFR1',  'FGFR2',
       'FGFR3',  'FLT3',    'GNA11',  'GNAQ',
       'GNAS',   'HRAS',    'IDH1',   'IDH2',
       'JAK2',   'JAK3',    'KIT',    'KLLN',
       'KRAS',   'MAP2K1',  'MET',    'MLH1',
       'MPL',    'MYC',     'NOTCH1', 'NRAS',
       'PDGFRA', 'PIK3CA',  'PIK3R1', 'PTEN',
       'PTPN11', 'RB1',     'RET',    'RUNX1',
       'SMAD4',  'SMARCB1', 'SRC',    'STK11',
       'TP53',   'VHL',     'WRAP53'
   ]) AS symbol
), luad AS (
   SELECT COUNT(DISTINCT sample_barcode_tumor) AS unique_samples
     FROM `isb-cgc.TCGA_hg38_data_v0.Somatic_Mutation` tcga_mut
    WHERE tcga_mut.sample_barcode_tumor IN (SELECT samplebarcode FROM `isb-cgc.tcga_cohorts.LUAD`)
)
  SELECT genes.symbol Hugo_Symbol,  
         COUNT(DISTINCT sample_barcode_tumor)/(SELECT unique_samples FROM luad) mut_freq
    FROM genes, `isb-cgc.TCGA_hg38_data_v0.Somatic_Mutation` tcga_mut
   WHERE genes.symbol = tcga_mut.symbol
     AND tcga_mut.Variant_Type = 'SNP'
     AND tcga_mut.sample_barcode_tumor IN (SELECT samplebarcode FROM `isb-cgc.tcga_cohorts.LUAD`)
GROUP BY genes.symbol
ORDER BY mut_freq DESC

In [ ]:
tcga

## Genie Query
Now query the same set of genes against the Genie tables.

In [ ]:
query = """
WITH genes AS (
  SELECT * FROM UNNEST([
       'ABL1',   'AKT1',    'ALK',    'APC',
       'BRAF',   'CDH1',    'CDKN2A', 'CSF1R',
       'CTNNB1', 'EGFR',    'ERBB2',  'ERBB3',
       'ESR1',   'FBXW7',   'FGFR1',  'FGFR2',
       'FGFR3',  'FLT3',    'GNA11',  'GNAQ',
       'GNAS',   'HRAS',    'IDH1',   'IDH2',
       'JAK2',   'JAK3',    'KIT',    'KLLN',
       'KRAS',   'MAP2K1',  'MET',    'MLH1',
       'MPL',    'MYC',     'NOTCH1', 'NRAS',
       'PDGFRA', 'PIK3CA',  'PIK3R1', 'PTEN',
       'PTPN11', 'RB1',     'RET',    'RUNX1',
       'SMAD4',  'SMARCB1', 'SRC',    'STK11',
       'TP53',   'VHL',     'WRAP53'
   ]) AS symbol
), luad AS (
  SELECT DISTINCT patient_id, sample_id
    FROM `project-genie-query-prod.consortium.sample` 
   WHERE cancer_type_detailed = 'Lung Adenocarcinoma'
), patient AS (
  SELECT COUNT(DISTINCT patient_id) total
    FROM `project-genie-query-prod.consortium.mutation` m, luad
   WHERE m.Tumor_Sample_Barcode = luad.sample_id
)
  SELECT m.Hugo_Symbol, 
         COUNT(DISTINCT luad.patient_id)/(SELECT total FROM patient) mut_freq
    FROM `project-genie-query-prod.consortium.mutation` m, luad
   WHERE m.Hugo_Symbol IN (SELECT symbol FROM genes)
     AND m.Variant_Type ='SNP'
     AND m.Tumor_Sample_Barcode = luad.SAMPLE_ID 
GROUP BY m.Hugo_Symbol 
ORDER BY mut_freq desc"""
query_job = client.query(
    query,
    location="US",
)  # API request - starts the query

genie_df = query_job.to_dataframe()
genie_df

## Merge Tables

In [ ]:
import pandas as pd
results = pd.merge(tcga, genie, on='Hugo_Symbol')

## Plot

In [ ]:
p = results.plot.scatter(x='mut_freq_x', y='mut_freq_y', grid=True)
p.set_title('GENIE vs TCGA % Mutated \n 50 Top GENIE Assay Covered Genes')
p.set_xlabel('TCGA % Mutated')
p.set_ylabel('GENIE % Mutated')
for i, txt in enumerate(results.Hugo_Symbol):
    if results.mut_freq_x.iat[i] > 0.09:
        p.annotate(txt, (results.mut_freq_x.iat[i] + 0.005, results.mut_freq_y.iat[i]))
p.annotate("",
              xy=(0, 0), xycoords='data',
              xytext=(0.4, 0.4), textcoords='data',
              arrowprops=dict(arrowstyle="-",
                              connectionstyle="arc3,rad=0."), 
              )